In [1]:
import numpy as np
import random as rn
import findspark
import sys
import time
import numba
from numba import cuda, float32, float64
#findspark.init("/opt/spark")
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [2]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [3]:
#PARÁMETROS del problema
V_max= 5.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 100

In [4]:
#Estructuas de datos
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum= sc.accumulator(0)
accum_ind= sc.accumulator(0)
mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)
broadcastInd= sc.broadcast(ind)

In [5]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    global accum
    parts_ =[]
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        velocidad=[rn.uniform(-5,5) for j in range(N)]
        part_ = posicion + velocidad + posicion
        parts_.append(part_) 
        fit = MSE(posicion,objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            mejor_pos_global = posicion
        accum= fit
    parts = np.array(parts_)
    return blf,bgf,mejor_pos_global,parts

In [6]:
def fitnessEval(part,mpg,blf,bgf,N):
    global accum
    global accum_ind
    global ind
    filas=part[0:N]
    fit = MSE(filas,objetivo)
    if fit < blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        for k in range(N):
            part[2*N+k]=filas[k]
        if fit < bgf:
            bgf = fit
            mpg = filas
    accum= fit
    if ind<10:
        ind+=1
    accum_ind= ind
    #return part,mpg,blf,bgf
    return part

In [7]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)
rdd= sc.parallelize(particulas)
print("particulas:", particulas)
print("particulas[0]", particulas[0])
print("particulas[1]", particulas[1])
print("accum ", accum)
#print("resultado=",rdd.map(lambda x:(x[0],x[1],x[2])).collect())
#print("best local fitness:",best_local_fitness)
#print("best global fitness:",best_global_fitness)
resultado=rdd.map(lambda x:fitnessEval(x,mejor_pos_global,best_local_fitness,best_global_fitness,n)).collect()
print("resultado ",resultado)
print("best global fitness:",accum)
print("valor final de ind:",accum_ind)

particulas: [[ 8.62184188e+01 -1.25513130e+00  2.36841885e+01  2.91095487e+00
   2.65839413e+00 -3.83434294e+00  8.62184188e+01 -1.25513130e+00
   2.36841885e+01]
 [ 5.59748032e+01 -3.61570399e+01 -9.52292630e+01 -4.24822141e+00
   7.10131369e-01  4.97829139e+00  5.59748032e+01 -3.61570399e+01
  -9.52292630e+01]
 [ 3.70158085e+01 -6.42010069e+01 -5.56903551e+01 -1.95824635e+00
   2.76236608e+00 -3.85354065e+00  3.70158085e+01 -6.42010069e+01
  -5.56903551e+01]
 [-5.16672082e+01 -8.51829089e+01  7.78100726e+01 -2.11533797e-02
   7.61904431e-01 -2.06009081e+00 -5.16672082e+01 -8.51829089e+01
   7.78100726e+01]
 [ 4.74898521e+01 -4.31754964e+01 -2.26239723e+00  3.43473405e+00
   3.47265698e+00 -9.24482968e-02  4.74898521e+01 -4.31754964e+01
  -2.26239723e+00]
 [ 5.77441229e+01  4.95630713e+01 -6.83359880e+01  1.81826584e+00
   1.57780296e+00 -2.52011831e+00  5.77441229e+01  4.95630713e+01
  -6.83359880e+01]
 [ 4.14043658e+01 -1.95013903e+01  6.60100914e+01 -8.87399796e-01
   1.82198978e+0